<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Packages" data-toc-modified-id="Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Packages</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Plan" data-toc-modified-id="Plan-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Plan</a></span></li><li><span><a href="#Brownian-motion" data-toc-modified-id="Brownian-motion-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Brownian motion</a></span></li><li><span><a href="#Monte-Carlo" data-toc-modified-id="Monte-Carlo-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Monte Carlo</a></span><ul class="toc-item"><li><span><a href="#UDF" data-toc-modified-id="UDF-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>UDF</a></span></li><li><span><a href="#Computations" data-toc-modified-id="Computations-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Computations</a></span></li><li><span><a href="#Previously-computed" data-toc-modified-id="Previously-computed-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Previously computed</a></span></li></ul></li><li><span><a href="#LETF" data-toc-modified-id="LETF-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>LETF</a></span><ul class="toc-item"><li><span><a href="#UDF" data-toc-modified-id="UDF-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>UDF</a></span></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Analysis</a></span></li></ul></li></ul></div>

## Packages

In [2]:
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt
import sys

## Data

We load prepared in xlsx file data for the underlyings TR

In [3]:
link = 'Data/Underlyings from ICE.xlsx'

In [4]:
Data = pd.read_excel(io=link, index_col=0, parse_dates=True)

# Calulate log difference
Data['logdif_7_10'] = np.log(Data.Under_7_10) - np.log(Data.Under_7_10.shift(1)) 
Data['logdif_20'] = np.log(Data.Under_20) - np.log(Data.Under_20.shift(1)) 

For the Brownian Motion, we will calculate mean/volatility

In [5]:
Parameters = {}

Parameters['All years'] = [Data.Under_7_10[-1],
                             Data.logdif_7_10.mean(),
                             Data.logdif_7_10.std()]

# Make horizon of 3 years
Data2 = Data.loc['2019-02-28':'2021-02-28']

Parameters['Three years'] = [Data2.Under_7_10[-1],
                             Data2.logdif_7_10.mean(),
                             Data2.logdif_7_10.std()]

In [6]:
Parameters

{'All years': [119.914, 0.00016162673084323253, 0.0035037171053943997],
 'Three years': [119.914, 0.0002516561380959002, 0.003814277497601597]}

## Plan

- Create Brownian motion, with simulation
- Create LETF and rebalance
- Create Monte Carlo

## Brownian motion

In [7]:
# UDF to generate error terms ( need for simple)
def Create_Errors(what,horizon):
    
    if what == 'Stand Norm':

        Stand_Residuals = [random.gauss(0,1) for x in range(horizon)]

    return Stand_Residuals

In [8]:
class Brownian_Motion(): # Currently for constant mu and sigma
    
    def __init__(self,Inital_Price, TimeDelta, mu, sigma):
        
        self.Inital_Price = float(Inital_Price)
        self.TimeDelta = int(TimeDelta)
        self.mu = float(mu)
        self.sigma = float(sigma)
        
    def Simulate_Prices(self,epsilons):
        
        predited_prices = []
        predited_prices.append([0,self.Inital_Price])
        
        for i, epsilon in enumerate(epsilons,start=1):
            
            if len(predited_prices) == 1:
                
                num1 = (self.mu - np.power(self.sigma,2)/2)*self.TimeDelta 
                num2 = self.sigma*epsilon*np.sqrt(self.TimeDelta)

                iPrice = self.Inital_Price*np.exp(num1-num2)

                predited_prices.append([i,iPrice])
                
            else:
                
                num1 = (self.mu - np.power(self.sigma,2)/2)*self.TimeDelta
                num2 = self.sigma*epsilon*np.sqrt(self.TimeDelta)

                iPrice = iPrice*np.exp(num1-num2)

                predited_prices.append([int(i),iPrice])
        
        predited_prices = pd.DataFrame(predited_prices)
        
        predited_prices.columns = ['day','price']
        
        predited_prices.set_index('day', inplace=True)
        
        return predited_prices
    
    def __repr__(self):
        return '''
Initial Price {0}
Time Delta {1}
Mu {2}
Sigma {3}
'''.format(self.Inital_Price,self.TimeDelta, self.mu,self.sigma)
        

## Monte Carlo

### UDF

In [9]:
def Monte_Carlo(Inital_Price, TimeDelta, mu, sigma,
                       n_simulations = 5000, 
                       error_type = 'Stand Norm', n_holding_days = 5):
    
    BM_i = Brownian_Motion(Inital_Price, TimeDelta, mu, sigma) # create brownian motion
    
    Simulated_DF = pd.DataFrame()
    
    for iSim in range(n_simulations): # For each simulation

        # create simulated errors
        Errors_i = Create_Errors(error_type,n_holding_days)

        # create returns of underlying for errors
        Simulation_i = BM_i.Simulate_Prices(Errors_i)
        
        # Rename column
        Simulation_i.columns = ['Price_'+str(iSim)]
        
        # Add data
        
        Simulated_DF = pd.concat([Simulated_DF, Simulation_i],axis=1)
        
    return Simulated_DF

### Computations

In [36]:
# I saved file, but code is rather effient -> 2 min 37 seconds to generate all needed data for 10000

Data_week = Monte_Carlo(Inital_Price=Parameters['Three years'][0],
                    TimeDelta=1,
                    mu=Parameters['Three years'][1],
                    sigma=Parameters['Three years'][2],
                    n_holding_days=5,
                    n_simulations = 10000)

Data_month = Monte_Carlo(Inital_Price=Parameters['Three years'][0],
                    TimeDelta=1,
                    mu=Parameters['Three years'][1],
                    sigma=Parameters['Three years'][2],
                    n_holding_days=21,
                    n_simulations = 10000)

Data_year = Monte_Carlo(Inital_Price=Parameters['Three years'][0],
                    TimeDelta=1,
                    mu=Parameters['Three years'][1],
                    sigma=Parameters['Three years'][2],
                    n_holding_days=252,
                    n_simulations = 10000)

Data_week.to_csv('Data/Simulated_data_week.csv')
Data_month.to_csv('Data/Simulated_data_month.csv')
Data_year.to_csv('Data/Simulated_data_year.csv')

### Previously computed

In [10]:
# I saved file, but code is rather effient -> 2 min 37 seconds to generate all needed data for 10000

Data_week = pd.read_csv('Data/Simulated_data_week.csv', index_col=0)
Data_month = pd.read_csv('Data/Simulated_data_month.csv', index_col=0)
Data_year = pd.read_csv('Data/Simulated_data_year.csv', index_col=0)

## LETF

### UDF

In [11]:
def Quick_LETF_Return(Index_return, Leverage, Fees=0):
    return Index_return*Leverage-Fees/252

In [12]:
def Create_LETF(Data,Leverage, Fees=0):
    
    NewData = Data.pct_change().fillna(0)

    NewData = NewData.apply(Quick_LETF_Return, args=(Leverage,Fees))

    NewData = NewData + 1

    NewData = NewData.cumprod()
    
    return NewData

In [13]:
# plot it

def plot_it(Simulated_Data, Name):
    
    plt.style.use('seaborn-ticks')
    
    fig, ax = plt.subplots(figsize=(16, 9))

    for iKey in Simulated_Data.columns:

        ax.plot(Simulated_Data[iKey], color = 'blue')

    
    ax.set(xlabel='Day', ylabel='Return',
           title='Simulation')
    ax.grid()

    fig.savefig("Graphs/{0}.png".format(str(Name)))
    
    plt.show()

In [14]:
# plot it

def plot_it_2(Simulated_Data,Simulated_Data_fee):

    fig, ax = plt.subplots(figsize=(16, 9))

    for iKey in Simulated_Data.columns:

        ax.plot(Simulated_Data[iKey], color = 'blue')

    for iKey in Simulated_Data_fee.columns:

        ax.plot(Simulated_Data_fee[iKey], color = 'red')

    
    ax.set(xlabel='Day', ylabel='NAV, comparative to innitial, in %',
           title='Simulation')
    ax.grid()

    plt.show()

### Analysis

In [47]:
table = pd.DataFrame(columns = ['leverage','period','min','max','median','5% percentile'])

for iLeverage, iFee in [(1,0.0015),(2,0.0095),(3,0.0109),(-2,0.0095),(-3,0.0107)]:
    
    for iPeriod,iData in [('week',Data_week),('month',Data_month),('year',Data_year)]:

        # need to create table
        iLETF = Create_LETF(iData,iLeverage,iFee)
        
        table = table.append({'leverage':iLeverage,
                      'period':iPeriod,
                       'min':iLETF.iloc[-1,].min(),
                       'max':iLETF.iloc[-1,].max(),
                       'median':iLETF.iloc[-1,].median(),
                       '5% percentile':iLETF.iloc[-1,].quantile(.05)},
                     ignore_index=True)

In [48]:
# save the results
table.to_csv('Data/Appendix_10.csv')

In [50]:
table.to_excel('Data/Appendix_10.xlsx', index=False)